In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
conn = sqlite3.connect('my_db')

In [3]:
def select(sql):
    return pd.read_sql(sql, conn)

### Создаем таблицу с водителями 'drivers':
- **driver_id** - уникальный номер водителя
- **district** - район, в котором находится водитель
- **active_flag** - бинарная переменная: 1 - водитель активен и может принимать заказы, 0 - неактивен, не принимает заказы
<br>

Таблица регулярно обновляется, перезаписывая переменную **district**, если водитель активен.

In [94]:
t = pd.DataFrame({'driver_id': [11, 22, 33, 44, 55, 66, 77, 88, 99, 100],
                 'district': ['a', 'b', 'a', 'c', 'a', 'd', 'b', 'c', 'b', 'a'],
                 'active_flag': [1, 1, 1, 0, 1, 1, 0, 0, 1, 1]
})

In [39]:
t.to_sql('drivers', conn, index=False, if_exists='replace')

10

In [103]:
sql = '''
select * from drivers t
'''
select(sql)

,driver_id,district,active_flag
0,11,a,1
1,22,b,1
2,33,a,1
3,44,c,0
4,55,a,1
5,66,d,1
6,77,b,0
7,88,c,0
8,99,b,1
9,100,a,1


### Создаем таблицу с заказами 'orders':
- **order_id** - уникальный номер заказа
- **date_time** - время заказа
- **district** - район, из которого пришел заказ
<br>

Таблица регулярно обновляется. Добавляются новые заказы.

In [95]:
tt = pd.DataFrame({'order_id': [1, 2, 3, 4, 5, 6, 7, 8],
                   'date_time': pd.to_datetime(['2023-09-01', '2023-09-01','2023-09-01', 
                                                '2023-09-01', '2023-09-01', '2023-09-01',
                                                '2023-09-01', '2023-09-01']),
                   'district': ['a', 'b', 'c', 'c', 'e', 'b', 'd', 'c'],
})

In [32]:
tt.to_sql('orders', conn, index=False, if_exists='replace')

8

In [104]:
sql = '''
select * from orders t
'''
select(sql)

,order_id,date_time,district
0,1,2023-09-01 00:00:00,a
1,2,2023-09-01 00:00:00,b
2,3,2023-09-01 00:00:00,c
3,4,2023-09-01 00:00:00,c
4,5,2023-09-01 00:00:00,e
5,6,2023-09-01 00:00:00,b
6,7,2023-09-01 00:00:00,d
7,8,2023-09-01 00:00:00,c


### Считаем кол-во водителей по районам. Присваиваем каждому району ранг, где 1 - наименьшее кол-во водителей.

In [101]:
sql = '''
with act_driver_count as
(select t.district,  sum(t.active_flag) as act_driver_count
from drivers t
group by t.district)

select t.*, dense_rank() over(order by t.act_driver_count) as rank
from act_driver_count t
'''
select(sql)

,district,act_driver_count,rank
0,c,0,1
1,d,1,2
2,b,2,3
3,a,4,4


### Джойним столбец 'rank' из таблицы 'drivers' к таблице 'orders', чтобы на основании ранга вывести наценку на заказ.
- Ранг 1 - наценка 1.3
- Ранг 2 - наценка 1.2
- Ранг 3 - наценка 1.1
- Остальные ранги - по обычному тарифу.

In [102]:
sql = '''
with act_driver_count as
(select t.district,  sum(t.active_flag) as act_driver_count
from drivers t
group by t.district),

district_rank as
(select t.*, dense_rank() over(order by t.act_driver_count) as rank
from act_driver_count t),

orders_rank as
(select t.*, r.rank from orders t
left join district_rank r on t.district = r.district)

select t.order_id, t.date_time, t.district,
case when t.rank = 1 then 1.3
     when t.rank = 2 then 1.2
     when t.rank = 3 then 1.1
     else 1 end as extra_coef
from orders_rank t
'''
select(sql)

,order_id,date_time,district,extra_coef
0,1,2023-09-01 00:00:00,a,1.0
1,2,2023-09-01 00:00:00,b,1.1
2,3,2023-09-01 00:00:00,c,1.3
3,4,2023-09-01 00:00:00,c,1.3
4,5,2023-09-01 00:00:00,e,1.0
5,6,2023-09-01 00:00:00,b,1.1
6,7,2023-09-01 00:00:00,d,1.2
7,8,2023-09-01 00:00:00,c,1.3
